In [23]:
import os
import asyncio
import nest_asyncio
nest_asyncio.apply()

SEASONS = list(range(2016,2024))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [24]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

In [25]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        await asyncio.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [26]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [27]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [28]:
async def main():
    tasks = []
    for season in SEASONS:
        tasks.append(scrape_season(season))
    await asyncio.gather(*tasks)

    standings_files = os.listdir(STANDINGS_DIR)
    
    tasks = []
    for season in SEASONS:
        files = [s for s in standings_files if str(season) in s]
        for f in files:
            filepath = os.path.join(STANDINGS_DIR, f)
            tasks.append(scrape_game(filepath))
    await asyncio.gather(*tasks)

In [29]:
asyncio.run(main())

Task exception was never retrieved
future: <Task finished name='Task-10' coro=<Connection.run() done, defined at C:\Users\bryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\playwright\_impl\_connection.py:269> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\bryan\AppData\Local\Programs\Python\Python311\Lib\asyncio\tasks.py", line 267, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "C:\Users\bryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\playwright\_impl\_connection.py", line 276, in run
    await self._transport.connect()
  File "C:\Users\bryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\playwright\_impl\_transport.py", line 135, in connect
    raise exc
  File "C:\Users\bryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\playwright\_impl\_transport.py", line 123, in connect
    self._proc = await asyncio.create_subprocess_exec(
                 ^^^^^^^^^^^^^^^^^^^^^

NotImplementedError: 